In [ ]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import copy

from pprint import pprint

import numpy as np

## The *Nim* and *Nimply* classes

In [ ]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = sys.maxsize) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def final_stage(self) -> bool:
        return self._finalstage

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert num_objects <= self._k
        self._rows[row] -= num_objects

## Functions for optimal strageies

In [ ]:
def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def advanced_nim_sum(state: Nim) -> int:
    mex = [x % (state._k + 1) for x in state.rows]
    #print(mex)
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in mex])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        if raw._k < ply.num_objects:
            continue
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        # cooked["possible_moves"][ply] = nim_sum(tmp)
        cooked["possible_moves"][ply] = advanced_nim_sum(tmp)
        
    return cooked

## Strategies

In [ ]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, min(state._k, state._rows[row]))
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns == 0]
    if not spicy_moves:
        ply = pure_random(state)
    else:
        ply = random.choice(spicy_moves)
        
    return ply

def agent(genotype) -> function:
    def agent_style(state: Nim) -> Nimply:
        probabilities = genotype[:N_STRATEGIES]
        probabilities /= sum(probabilities)
        chosen_strategy = np.random.choice(len(probabilities), num_samples, p=probabilities)
        
        ply = strategy_mapping[chosen_strategy]
        return ply
    return agent_style

strategy_mapping = {
    0: pure_random,
    1: gabriele,
    2: optimal,
}

## Evolutionary parameters

In [ ]:
POPULATION_SIZE = 30
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .15
N_STRATEGIES = 3
TOT_N_OFFSPRINGS = 1e5
λ = 20
σ = 1
τ = 1/np.sqrt(N_STRATEGIES)

TRAINING_GAMES = 100
ROWS = 5

## Evolutionary methods

In [ ]:
@dataclass
class Individual:
    fitness: float
    genotype: list[float]

def select_parent(pop) -> Individual:
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    # Mutate σ of the correspoding strategies
    offspring[N_STRATEGIES:] = offspring[N_STRATEGIES:]*np.exp(τ*np.random.normal(loc=0, scale=1))

    # Mutate the strategies "probabilities" with corresponding σ
    offspring[:N_STRATEGIES] = (
        np.random.normal(loc=offspring[:N_STRATEGIES], scale=offspring[N_STRATEGIES:].reshape(N_STRATEGIES, 1)) + ind
    )
    offspring.fitness = None
    return offspring

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, N_STRATEGIES-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:N_STRATEGIES] + ind1.genotype[N_STRATEGIES:])
    assert len(offspring.genotype) == N_STRATEGIES
    return offspring

def fitness(genotype) -> float:
    # Make the individual play
    wins = 0
    for game in tqdm(range(TRAINING_GAMES)):
        nim = Nim(ROWS)
        player = np.random.choice([0, 1])
        strategy_int = np.random.choice(np.array([i for i in range(N_STRATEGIES)]))
        strategy = (agent(genotype), strategy_mapping[strategy_int])

        while nim:
            ply = strategy[player](nim)
            #print(f"ply: player {player} plays {ply}")
            nim.nimming(ply)
            #print(f"status: {nim}")
            player = 1 - player
            
        if player == 1:
            wins += 1
        
    return wins/TRAINING_GAMES


## Generating strating population

In [ ]:
population = [
    Individual(
        # genotype=[choice((False, False)) for _ in range(NUM_SETS)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]

## Evolutionary algorithm

In [ ]:
for step in tqdm(range(TOT_N_OFFSPRINGS // λ)):
    # Generate offsprings
    offspring = list()
    for counter in range(λ):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  # add more clever mutations
            p = select_parent(population)
            o = mutate(p)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = one_cut_xover(p1, p2)
        offspring.append(o)

    for i in offspring:
        i.fitness = fitness(i.genotype)

    # Selection of survivals    
    population.extend(offspring)
    population.sort(key=lambda i: i.fitness, reverse=True)
    population = population[:POPULATION_SIZE]
    
    print(f"individual:{population[0][:N_STRATEGIES]}, fitness={population[0].fitness}")